In [1]:
import geopandas as gpd
import shapely
from shapely.ops import unary_union
from bc_power import utils, hydro
import pandas as pd
from shapely.geometry import Point
import pypsa
import numpy as np
import atlite
import pyomo
import matplotlib.pyplot as plt
import math
# from pypsa.networkclustering import get_clustering_from_busmap
import matplotlib.dates as mdates
import cartopy.crs as ccrs

pyomo.__version__

'6.5.0'

In [100]:
def add_pypsa_dict(network, comp_list):
    for comp_dict in comp_list:
        if comp_dict['bus'] == 'CentralCoast':
            continue
        if comp_dict['bus'] == 'Stikine':
            continue
        if comp_dict['bus'] == "NorthernRockies":
            continue
        network.add(**comp_dict)


In [77]:
network_path = "/home/pmcwhannel/repos/PyPSA_BC/results/tester.nc" # cfg['pypsa']['results']
n = pypsa.Network(override_component_attrs=utils.get_multi_link_override())
pypsa.Network.import_from_netcdf(network=n, path=network_path)

INFO:pypsa.io:Imported network tester.nc has buses, carriers, generators, lines, line_types, links, loads, stores, transformer_types


In [60]:
geojson_file = "/home/pmcwhannel/repos/PyPSA_BC/data/regions/gadm41_CAN_1.json"
gdf = gpd.read_file(geojson_file)

In [67]:
mask = gdf["NAME_1"] == "BritishColumbia"
gdf.loc[mask,:].geometry.centroid.y.iloc[0]

/tmp/ipykernel_24547/1688874521.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf.loc[mask,:].geometry.centroid.y.iloc[0]


54.75339738834036

In [58]:
n.buses

,x,y,type,v_nom,substation_type,carrier,control,generator,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,sub_network
Bus,,,,,,,,,,,,,
138_BCAB3_IPT,-114.200000,50.247937,138kV,138.0,IPT,AC,PQ,,,1.0,0.0,inf,
138_BCAB1_IPT,-114.600000,49.735535,138kV,138.0,IPT,AC,PQ,,,1.0,0.0,inf,
138_BCAB4_IPT,-119.700000,58.645250,138kV,138.0,IPT,AC,Slack,,,1.0,0.0,inf,
230_BCUS2_INT,-117.341514,48.997400,230kV,230.0,INT,AC,PQ,,,1.0,0.0,inf,
500_BCUS1_INT,-122.873948,48.970000,500kV,500.0,INT,AC,PQ,,,1.0,0.0,inf,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Kitimat-Stikine,-129.457201,55.740296,,300.0,,AC,PQ,,,1.0,0.0,inf,
Bulkley-Nechako,-125.738993,54.542789,,300.0,,AC,PQ,,,1.0,0.0,inf,
Cariboo,-123.102207,52.343426,,300.0,,AC,PQ,,,1.0,0.0,inf,


### V2G EV Integration

In [36]:
network_path = "/home/pmcwhannel/repos/PyPSA_BC/results/network_debug_cyclic_backstop.nc" # cfg['pypsa']['results']
n = pypsa.Network(override_component_attrs=utils.get_multi_link_override())
pypsa.Network.import_from_netcdf(network=n, path=network_path)

INFO:pypsa.io:Imported network network_debug_cyclic_backstop.nc has buses, carriers, generators, lines, line_types, links, loads, stores, transformer_types


In [37]:
charge_strat = "v2g"
prefix = "/home/pmcwhannel/repos/PyPSA_BC/data/"
ev_bus = utils.read_pickle(prefix + "ev/{}_ev_bus.pickle".format(charge_strat))
ev_load = utils.read_pickle(prefix + "ev/{}_ev_load.pickle".format(charge_strat))
ev_battery = utils.read_pickle(prefix + "ev/{}_ev_battery.pickle".format(charge_strat))
ev_charger = utils.read_pickle(prefix + "ev/{}_ev_charger.pickle".format(charge_strat))
ev_discharger = utils.read_pickle(prefix + "ev/{}_ev_discharger.pickle".format(charge_strat))


In [38]:
for component in ev_bus:
    name = component['name'].split('_')[2]
    c1 = name != "CentralCoast"
    c2 = name != "Stikine"
    c3 = name != "NorthernRockies"
    if c1 and c2 and c3:
        n.add(**component)

for component in ev_load:
    name = component['name'].split('_')[0]
    c1 = name != "CentralCoast"
    c2 = name != "Stikine"
    c3 = name != "NorthernRockies"
    if c1 and c2 and c3:
        n.add(**component)

for component in ev_battery:
    name = component['name'].split('_')[2]
    c1 = name != "CentralCoast"
    c2 = name != "Stikine"
    c3 = name != "NorthernRockies"
    if c1 and c2 and c3:
        n.add(**component)

for component in ev_charger:
    name = component['bus0']
    c1 = name != "CentralCoast"
    c2 = name != "Stikine"
    c3 = name != "NorthernRockies"
    if c1 and c2 and c3:
        n.add(**component)

for component in ev_discharger:
    name = component['bus1']
    c1 = name != "CentralCoast"
    c2 = name != "Stikine"
    c3 = name != "NorthernRockies"
    if c1 and c2 and c3:
        n.add(**component)

/home/pmcwhannel/anaconda3/envs/bc-power/lib/python3.9/site-packages/pypsa/components.py:838: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cls_pnl[k][name] = pd.Series(data=v, index=self.snapshots, dtype=typ)
/home/pmcwhannel/anaconda3/envs/bc-power/lib/python3.9/site-packages/pypsa/components.py:838: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cls_pnl[k][name] = pd.Series(data=v, index=self.snapshots, dtype=typ)
/home/pmcwhannel/anaconda3/envs/bc-power/lib/python3.9/site-packages/pypsa/components.py:838: PerformanceWarning: 

### Testing uncoordinated EV integration

In [43]:
network_path = "/home/pmcwhannel/repos/PyPSA_BC/results/tester_uncoordinated.nc" # cfg['pypsa']['results']

n = pypsa.Network(override_component_attrs=utils.get_multi_link_override())
ev_load_list = utils.read_pickle("/home/pmcwhannel/repos/PyPSA_BC/data/ev/uncoordinated_ev_load.pickle")
pypsa.Network.import_from_netcdf(network=n, path=network_path)


# Add backstop tecnology
# for bus in n.loads.bus.unique():
#     n.add(class_name="Generator",
#                 name='Backstop {}'.format(bus),
#                 bus=bus,
#                 p_nom=0,
#                 marginal_cost=1000,
#                 p_nom_extendable=True
#                 )
    
# add_pypsa_dict(n, ev_load_list)

INFO:pypsa.io:Imported network tester_uncoordinated.nc has buses, carriers, generators, lines, line_types, links, loads, stores, transformer_types


In [52]:
mask = ~ n.loads.index.str.contains("ELC Load")
cols = n.loads[mask].index

In [56]:
n.loads_t.p_set[cols].sum(axis=1).sum()

9270099.654530497

In [116]:
n.optimize(solver_name='gurobi')

/home/pmcwhannel/anaconda3/envs/bc-power/lib/python3.9/site-packages/numpy/core/numeric.py:407: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(res, fill_value, casting='unsafe')
INFO:linopy.model: Solve linear problem using Gurobi solver
Writing variables.: 100%|██████████| 6/6 [00:02<00:00,  2.92it/s]


Read LP format model from file /tmp/linopy-problem-f60cz0lr.lp
Reading time = 11.50 seconds
obj: 10196415 rows, 3372625 columns, 18176518 nonzeros
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: 12th Gen Intel(R) Core(TM) i7-1270P, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 10196415 rows, 3372625 columns and 18176518 nonzeros
Model fingerprint: 0x2761d27f
Coefficient statistics:
  Matrix range     [8e-05, 9e+03]
  Objective range  [1e-06, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-04, 1e+15]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 9679576 rows and 2002190 columns (presolve time = 5s) ...
Presolve removed 9734035 rows and 2100455 columns
Presolve time: 9.43s
Presolved: 462380 rows, 1272170 columns, 2593024 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, a

: 

In [107]:
cols = n.generators[n.generators.index.str.contains("Backstop")].index